# Raster Statistics

Runs the raster statistics on the Burn Area Index (BAI) rasters by creating a point grid from the merged raster images.

## Setting the Libraries

In [ ]:
import os
import rasterio
from rasterio.merge import merge
from shapely.geometry import mapping, Polygon, Point
from rasterio import features
import fiona
from fiona.crs import from_epsg
from osgeo import gdal
import ogr, osr
from osgeo import ogr
import csv
import geojson
import json
import pandas
from rasterstats import raster_stats

## Setting the path

In [ ]:
inpath = "/Users/Jenny/Document/Other/Fire_Project/GIS/"

## Running the rest of the code

In [ ]:
# Function that merges all the bai rasters as preparation for creating the point grid
def merge_all_rast(outpath, inpath):
    # Blank list to be appended to
    files_mosaic = []
    
    # For loop to access all files that meet the criteria of ending in "bai.tif"
    for file in inpath:
        src = rasterio.open(file)
        files_mosaic.append(src)
    
    # Merge function for the files set to the above list
    mosaic, out_trans = merge(files_mosaic)
    
    # Collects the metadata of the original bai rasters
    out_meta = src.meta.copy()
    
    # Writes out the merged raster
    with rasterio.open(outpath, "w", **out_meta) as dest:
        dest.write(mosaic)
        
# Selects the files using a unique identifier and sets up the out paths
def compare():
    ending_list = []
    for root, dirs, files in os.walk(inpath):
        for filename in files:
            if filename.endswith("bai.tif"):
                print('File name is: {0}'.format(filename))
                ending_list.append(filename)
                outpath = inpath + "all_merge.tif"
                os.chdir(inpath)
                merge_all_rast(outpath, ending_list)
            else:
                print("N/A")

compare()

# Outpath for the vect_shape function
outpath1 = os.path.join(inpath + "vect_shape.shp")

# Function to vectorize the merged raster from the merge_all_rast() function
def vect_shape(outpath, inpath):
    merge = os.path.join(inpath + "all_merge.tif")
    
    # Opens the merged raster and collects the bounds of the image
    raster = rasterio.open(merge)
    raster_bounds = tuple(raster.bounds)


    # define the lower and upper limits for x and y
    minX, maxX, minY, maxY = raster_bounds[0], raster_bounds[1], raster_bounds[2], raster_bounds[3]
    # create one-dimensional arrays for x and y

    print(raster_bounds)

    top_left = [minX, maxY]
    top_right= [minY, maxY]
    bottom_left = [minX, maxX]
    bottom_right = [minY, maxX]
    
    # Takes the above coordinates from the bounds and sets them as the new shapefile's bounds
    coords = [[top_left], [bottom_left], [top_right], [bottom_right]]

    poly = Polygon([top_left, top_right, bottom_right, bottom_left])
    
    # Schema necessary to write out the shapefile as a polygon
    schema = {
        'geometry': 'Polygon',
        'properties': {'id': 'int'},
    }
    
    # Writes the shapefile out as 4326 with the above criteria
    with fiona.open(outpath, 'w', crs=from_epsg(4326), driver ='ESRI Shapefile', schema = schema) as c:
        ## If there are multiple geometries, put the "for" loop here
        c.write({
            'geometry': mapping(poly),
            'properties': {'id': 123},
        })
vect_shape(outpath1, inpath)

# Outpath for the blank_rast function 
outpath2 = os.path.join(inpath + "blank_rast.tif")

# Function to create a blank raster from the shapefile created above
def blank_rast(outpath, inpath):
    vector_file = os.path.join(inpath + "vect_shape.shp")
    
    # Opens the vectorized merge file to create the blank raster
    shapefile = fiona.open(vector_file)
    geom = [shapes['geometry'] for shapes in shapefile]
    attrib = [shapes['properties'] for shapes in shapefile]


    #get metadata for satellite imagery
    merge = os.path.join(inpath + "all_merge.tif")
    rst = rasterio.open(merge)
    
    # Rasterizes the shapefile
    image = features.rasterize(geom, out_shape=rst.shape, transform=rst.transform, dtype = 'float64')
    
    # Writes out the new blank raster
    with rasterio.open(
            outpath , 'w',
            driver='GTiff',
            transform = rst.transform,
            dtype= 'float64',
            crs= 'EPSG:4326',
            count=1,
            width=rst.width,
            height=rst.height) as dst:
        dst.write(image, indexes=1)
blank_rast(outpath2, inpath)

# Outpath for the csv_pt function 
csv_outpath = os.path.join(inpath + "rast_coord.csv")

# Function to create a csv of coordinates from the centroids of the blank raster
def csv_pt(outpath, inpath):
    # Open tif file
    blank = os.path.join(inpath + "blank_rast.tif")
    ds = gdal.Open(blank)
    # GDAL affine transform parameters, According to gdal documentation xoff/yoff 
    #are image left corner, a/e are pixel wight/height and b/d is rotation and is 
    # zero if image is north up. 
    xoff, a, b, yoff, d, e = ds.GetGeoTransform()

    def pixel2coord(x, y):
    #"""Returns global coordinates from pixel x, y coords"""
        xp = a * x + b * y + xoff


        yp = d * x + e * y + yoff
        return(xp, yp)

    # get columns and rows of your image from gdalinfo
    cols = ds.RasterXSize + 1
    rows = ds.RasterYSize + 1
    print(cols)
    print(rows)
    # Output and export
    file = open(outpath,"w")
    for row in  range(0,rows):
        for col in  range(0,cols): 
            x,y = pixel2coord(col,row)
            file.write("{},{}\n".format(x,y))
csv_pt(csv_outpath, inpath)

# Outpath for the csv_pt function
json_outpath = os.path.join(inpath + "points.geojson")

# Function to write out csv coordinates as a geojson
def point_json(outpath, inpath):
    csv_path = os.path.join(inpath + "rast_coord.csv")
    
    # Opens the csv as data frame to convert to geojson
    data_frame = pandas.read_csv(csv_path, sep=',', names = ['lat','lon'])
    json_result_string = data_frame.to_json(
        orient='records'
    )
    json_result = json.loads(json_result_string)
    
    # Schema for the new geojson file
    geojson = {
        'type': 'FeatureCollection',
        'features':[]
    }
    for record in json_result:
        geojson['features'].append({
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': [record['lat'], record['lon']],
            },
            'properties': record,
        })
    
    # Writes out the file as a geojson
    with open(outpath, 'w') as out:
        out.write(json.dumps(geojson, indent=2))
point_json(json_outpath, inpath)


# Function to calculate the zonal statistics for each of the original bai images
def raster_stat(outpath, inpath):
    
    # Runs the raster stats function
    stats = raster_stats("points.geojson", inpath, stats="sum", copy_properties=True, nodata=0, geojson_out=True)
    result = {"type": "FeatureCollection", "features": stats}
    
    # Writes out the raster stats for each of the bai raster into a geojson
    with open(outpath, 'w') as outfile:
        geojson.dump(result, outfile)

# Function to search through folders to select files to run raster statistics
def compare_stat():
    ending_list = []
    
    # Searches for files that end with merge.tif and sets them to a new list
    for root, dirs, files in os.walk(inpath):
        for filename in files:
            if filename.endswith("bai.tif"):
                print('File name is: {0}'.format(filename))
                ending_list.append(filename.split('_',1)[0])
    
    # Creates a unique list from above
    unique_ending = list(set(ending_list))
    unique2 = unique_ending
    
    # Searches through the unique list to select files that are unique and contain the words bai
    for unique in unique_ending:
        for root, dirs, files in os.walk(inpath):
            for filename in files:
                if filename.startswith(unique):
                    files_unique = [x for x in files if x.startswith(unique)]
                    files_path = [x for x in files_unique if x.endswith("bai.tif")]
                    zonal_outpath = inpath + unique + "_zonstat.geojson"
                    if unique != False:
                        os.chdir(inpath)
                        raster_stat(zonal_outpath, files_path[0])
                else:
                    print("N/A")

compare_stat()